In [52]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate, TimeDistributed
from tensorflow.keras.layers import AdditiveAttention as Attention
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [53]:
# Function to preprocess data
def data_preprocessor(source_sentences, target_sentences):
    source_tokenizer = Tokenizer()
    source_tokenizer.fit_on_texts(source_sentences)
    source_sequences = source_tokenizer.texts_to_sequences(source_sentences)
    source_padded = pad_sequences(source_sequences, padding='post')
    
    target_tokenizer = Tokenizer()
    target_tokenizer.fit_on_texts(target_sentences)
    target_sequences = target_tokenizer.texts_to_sequences(target_sentences)
    target_padded = pad_sequences(target_sequences, padding='post')
    
    return source_padded, target_padded, source_tokenizer, target_tokenizer

# Load the training and test datasets
train_data = pd.read_csv(r'Z:\Projects\NLP\RussianTOEnglish\data\train.csv')  # Replace with the actual path to the training CSV
test_data = pd.read_csv(r'Z:\Projects\NLP\RussianTOEnglish\data\train.csv')  # Replace with the actual path to the test CSV

train_data = train_data[:1000]
test_data = test_data[:1000]

# Extract the 'rus' and 'eng' columns
train_source_sentences = train_data['rus'].values
train_target_sentences = train_data['eng'].values

test_source_sentences = test_data['rus'].values
test_target_sentences = test_data['eng'].values

# Preprocess the training data
train_input_texts, train_target_texts, source_tokenizer, target_tokenizer = data_preprocessor(train_source_sentences, train_target_sentences)

# Preprocess the test data (optional, for later testing)
test_input_texts, test_target_texts, _, _ = data_preprocessor(test_source_sentences, test_target_sentences)


In [54]:
# Model parameters
embedding_dim = 256
latent_dim = 256
num_encoder_tokens = len(source_tokenizer.word_index) + 1
num_decoder_tokens = len(target_tokenizer.word_index) + 1

In [55]:
# Encoder
encoder_inputs = Input(shape=(None,))  # Define the input layer for the encoder
encoder_embedding = Embedding(num_encoder_tokens, embedding_dim)(encoder_inputs)  # Apply embedding layer
encoder_lstm = LSTM(latent_dim, return_state=True)  # Initialize LSTM layer with latent dimension
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)  # Process embedded inputs through LSTM
encoder_states = [state_h, state_c]  # Store the states of the LSTM for use in the decoder

# Decoder
decoder_inputs = Input(shape=(None,))  # Define the input layer for the decoder
decoder_embedding = Embedding(num_decoder_tokens, embedding_dim)(decoder_inputs)  # Apply embedding layer
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)  # Initialize LSTM layer for decoder
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)  # Process inputs with LSTM

# Attention Layer
attention = Attention()  # Create an attention layer
attention_output = attention([decoder_outputs, encoder_outputs])  # Calculate attention output based on decoder and encoder outputs


In [56]:
# Concatenating attention output and decoder LSTM output 
decoder_concat_input = Concatenate(axis=-1)([decoder_outputs, attention_output])

# Reduce the dimensionality to 512 (or whatever is necessary)
decoder_projection = TimeDistributed(Dense(latent_dim, activation='relu'))(decoder_concat_input)

# Final Dense layer for softmax predictions
decoder_dense = TimeDistributed(Dense(num_decoder_tokens, activation='softmax'))
decoder_outputs = decoder_dense(decoder_projection)

In [57]:
# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert target sequences to one-hot encoding
train_decoder_target_data = to_categorical(train_target_texts, num_decoder_tokens)

# Train the model
model.fit([train_input_texts, train_target_texts], train_decoder_target_data, batch_size=64, epochs=1000, validation_split=0.2)

Epoch 1/1000
25/25 [==============================] - 4s 46ms/step - loss: 3.5819 - accuracy: 0.5466 - val_loss: 2.7677 - val_accuracy: 0.5811
Epoch 2/1000
25/25 [==============================] - 0s 17ms/step - loss: 2.6815 - accuracy: 0.5790 - val_loss: 2.6817 - val_accuracy: 0.5825
Epoch 3/1000
25/25 [==============================] - 0s 17ms/step - loss: 2.5380 - accuracy: 0.5844 - val_loss: 2.5481 - val_accuracy: 0.5936
Epoch 4/1000
25/25 [==============================] - 0s 16ms/step - loss: 2.2821 - accuracy: 0.6039 - val_loss: 2.2699 - val_accuracy: 0.6041
Epoch 5/1000
25/25 [==============================] - 0s 16ms/step - loss: 1.9992 - accuracy: 0.6369 - val_loss: 1.9767 - val_accuracy: 0.6486
Epoch 6/1000
25/25 [==============================] - 0s 15ms/step - loss: 1.8046 - accuracy: 0.6593 - val_loss: 1.8884 - val_accuracy: 0.7011
Epoch 7/1000
25/25 [==============================] - 0s 15ms/step - loss: 1.6394 - accuracy: 0.6934 - val_loss: 1.7699 - val_accuracy: 0.7139

In [58]:
# Encoder Inference Model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder Inference Model
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
# Ensure the start token is in the tokenizer
start_token = 'start'
if start_token not in target_tokenizer.word_index:
    target_tokenizer.word_index[start_token] = len(target_tokenizer.word_index) + 1

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [61]:
# Translation function
def translate(input_text):
    # Tokenize and pad the input sequence
    input_seq = source_tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=train_input_texts.shape[1], padding='post')
    print(f"input_seq shape: {input_seq.shape}")

    # Get the encoder states
    states_value = encoder_model.predict(input_seq)
    print(f"states_value shape: {[state.shape for state in states_value]}")

    # Generate an empty target sequence of length 1
    target_seq = np.zeros((1, 1))
    print(f"target_seq shape: {target_seq.shape}")

    # Populate the first character of the target sequence with the start character
    target_seq[0, 0] = target_tokenizer.word_index[start_token]  # Using <start> token

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_tokenizer.index_word.get(sampled_token_index, '')

        # Append the sampled token to the output sentence
        decoded_sentence += ' ' + sampled_char

        # Exit condition: either hit max length or find <end> token.
        if (sampled_char == '<end>' or len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence.strip()

# Translate a test sentence
translated_sentence = translate(test_source_sentences[5])
print(translated_sentence)
print(test_source_sentences[5])
print(test_target_sentences[5])


input_seq shape: (1, 5)
1/1 [==============================] - 0s 11ms/step
states_value shape: [(1, 256), (1, 256)]
target_seq shape: (1, 1)
1/1 [==============================] - 0s 14ms/step
soon evening evening concert concert concert please
давай сделаем это позже
let's do this later
